# Leren: Programming assignment 5
** This assignment can be done in teams of 2 **

**Student 1:**  <span style="color:red">de Groot</span> (<span style="color:red">10434410</span>)<br>
**Student 2:** <span style="color:red"> Dijkzeul</span> (<span style="color:red">10554386</span>)<br>

-----------------------------------
You may want to use parts of your code from the previous assignment(s) as a starting point for this assignment. 

The code you hand-in should follow the structure from this document. Write down your functions in the cells they belong to. Note that the structure corresponds with the structure from the actual programming assignment. Make sure you read this for the full explanation of what is expected of you. 

**Submission:**

* Make sure your code can be run from top to bottom without errors.
* Include your data files in the zip file.
* Comment your code

One way be sure you code can be run without errors is by quiting iPython completely and then restart iPython and run all cells again (you can do this by going to the menu bar above: Cell > Run all). This way you make sure that no old definitions of functions or values of variables are left (that your program might still be using).

-----------------------------------

If you have any questions ask your teaching assistent. We are here for you.

-----------------------------------

In [1]:
import pandas as pd
import numpy as np
import math

Training = pd.read_csv('digist123-1.csv', sep=";", header=None)
Test = pd.read_csv('digist123-2.csv', sep=";", header=None)

# Make the pandas dataframes numpy arrays
TrainingNpArray = np.array(Training)
TestNpArray = np.array(Test)

Training.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,0,0,1,11,15,8,0,0,0,0,...,0,0,0,2,15,15,3,0,0,1
1,0,0,4,10,11,4,0,0,0,1,...,0,0,0,5,12,12,12,1,0,1
2,0,0,5,14,12,5,0,0,0,0,...,0,0,0,7,13,16,8,0,0,1
3,0,0,14,10,0,0,0,0,0,0,...,0,0,0,12,16,16,16,16,9,1
4,0,0,5,12,10,4,0,0,0,0,...,0,0,0,8,12,12,4,0,0,1


In [2]:
# Make dict of all the classes
from collections import defaultdict

def makeClassDict(nparray):
    classDict = defaultdict(list) 
    for i in nparray:
        classDict[i[-1]].append(i[:-1])
    return classDict


In [3]:
def meanStdAll(classDict):
    allClasses = defaultdict(list) 
    for j in classDict.keys():
        for i in zip(*classDict[j]):
            if np.std(i) == 0:
                allClasses[j].append((np.mean(i),1))
            else:
                 allClasses[j].append((np.mean(i),np.std(i)))
    return allClasses


In [4]:
import math
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

In [5]:
def classProbs(allClasses, testData):
    probs = {}
    for keys in allClasses.keys():
        probs[keys] = 1
        for idx, i in enumerate(allClasses[keys]):
            mean, stdev = i
            x = testData[idx]
            probs[keys] *= calculateProbability(x, mean, stdev)
    return probs



In [6]:
classDict = makeClassDict(TrainingNpArray)

allClasses = meanStdAll(classDict)

# print predict(allClasses, TestNpArray[0])


def predict(allClasses, vector):
    probs = classProbs(allClasses, vector)
    best = [0, ""]
    for i in probs.keys():
        if probs[i] > best[0]:
            best[0] = probs[i]
            best[1] = i
    return best


In [7]:

# testData[:,:-1]
def getPredictions(allClasses, testData):
    predicts = []
    for i in testData:
        predicts.append(predict(allClasses, i)[1])
    return predicts

    

In [8]:
def accuracy(array1, array2):
    same = 0
    for i in range(len(array1)):
        if array1[i] == array2[i]:
            same += 1
    return same/float(len(array1))

In [9]:


classDict = makeClassDict(TrainingNpArray)

allClasses = meanStdAll(classDict)

predicts = getPredictions(allClasses, TrainingNpArray[:,:-1])

print accuracy(predicts, TrainingNpArray[:,-1])




0.986666666667


### 1b ###

The id's of the errors are 108, 144, 245, 279.
All these errors are either a 2 or a 3, and are predicted a 2 or a 3.
There is a very small difference between the two probabilities. 2 and 3 are very similar numbers and these the wrongly guessed 2 are probably very close to a 3 and vice versa.

To overcome this issue it is possible to better train the model. It is also possible to make an check which states that if the difference is so small, that it cannot predict a class with certainty. 


In [10]:
print predicts[108],TrainingNpArray[108][-1]

print classProbs(allClasses, TrainingNpArray[108][:-1])


print predicts[144],TrainingNpArray[144][-1]

print classProbs(allClasses, TrainingNpArray[144][:-1])


print predicts[245],TrainingNpArray[245][-1]

print classProbs(allClasses, TrainingNpArray[245][:-1])


print predicts[279],TrainingNpArray[279][-1]

print classProbs(allClasses, TrainingNpArray[279][:-1])




3 2
{1: 2.0149199278462528e-84, 2: 1.961669844242521e-64, 3: 2.1004351443462985e-63}
3 2
{1: 1.2157039581893253e-167, 2: 1.5132397685230444e-60, 3: 1.95278012433835e-56}
2 3
{1: 3.8483443456139399e-244, 2: 2.5159492972066129e-78, 3: 2.7040364741429635e-79}
2 3
{1: 4.0827522383018512e-189, 2: 1.1824230553825879e-63, 3: 2.61713342230593e-64}


### 2

In [47]:
import matplotlib.pyplot as plt

# Code made in week 3 for logisticregression


# uses the sigmoid function to calculate the hypothesis
def hypothesis(theta, xVector):
    return 1.0/(1.0+math.e **(-np.dot(theta,xVector)))

# Gradient function for the logistic regression
def gradientLog(theta,xMatrix,yVector, thetaIndex):
    gradient = (1.0/(1+math.e **(np.dot(theta,xMatrix))))-yVector
    gradient = np.dot(gradient,xMatrix[thetaIndex])
    return gradient/len(yVector)
    
# Update the theta vector for logistic regression
def updateLog(alpha, theta, xMatrix, yVector, Lambda):
    regularizationVector = [1]*len(theta)
    tempTheta = [None]*len(theta)
    for i in range(len(theta)):
        regularizationVector[i] *= Lambda/len(yVector)*theta[i]
    regularizationVector[0] = 0
    for i in range(len(theta)):
        tempTheta[i] = theta[i] - alpha * (gradientLog(theta,xMatrix,yVector,i) + regularizationVector[i])
    return tempTheta

# Calculate the cost for a logistic regrssion function
def costLogFunction(theta, xMatrix, yVector, Lambda):
    costSum = 0.0
    regularization = 0.0
    for i in range(len(yVector)):
        if yVector[i]:
            costSum += math.log(hypothesis(theta,xMatrix[i]))
        else:
            costSum += math.log(1-hypothesis(theta,xMatrix[i]))
            # dit geeft log(0) omdat hypothesis altijd 1 returned
            # dat mag niet
    for i in range(1,len(theta)):
        regularization += theta[i] **2
    return -(costSum/len(yVector) + (2*Lambda/len(yVector)*regularization))
    
# split y and multiply theta when multi-class classifying
def splitClasses(theta, yVector):
    classes = list(set(yVector))
    if len(classes) <=2 and 0 in classes:
        return theta, yVector
    else:
        # split y in het aantal klasses
        yMatrix = np.zeros((len(yVector),len(classes)))
        thetaMatrix = np.zeros((len(classes),len(theta)))
        for i in range(len(classes)):
            for j in range(len(yVector)):
                if yVector[j] == classes[i]:
                    yMatrix[j][i] = 1
                else:
                    yMatrix[j][i] = 0
            thetaMatrix[i] = theta
        yMatrix = yMatrix.T
        return thetaMatrix, yMatrix
            
def logreg(Digits, Lambda):

    alpha = 0.00001
    iterations = 120
    # Echt geen idee welke waarde Lambda moet hebben. Het werkt nu nog steeds dus maybe is het goed?
    yVector = Digits[Digits.columns[-1]]
    xMatrix = np.ones((Digits.columns[-1]+1,len(yVector)))
    theta = np.ones((Digits.columns[-1]+1))*0.0001
    for i in range(1,Digits.columns[-1]):
        xMatrix[i] = Digits[i]
        
    thetaMatrix, yMatrix = splitClasses(theta, yVector)
    
    costs = [[]]
    for i in range(len(yMatrix)):
        costs[0].append(costLogFunction(thetaMatrix[i], xMatrix.T, yMatrix[i], Lambda))
    #print "Old costs: ", costs[0]
    
    for i in range(iterations):
        costs.append([])
        for j in range(len(yMatrix)):
            thetaMatrix[j] = updateLog(alpha, thetaMatrix[j], xMatrix, yMatrix[j], Lambda)
            costs[i+1].append(costLogFunction(thetaMatrix[j], xMatrix.T, yMatrix[j], Lambda))
    
    #print "New cost: ", costs[-1]
    costs = np.array(costs).T
    x = np.arange(0,len(costs[0]))
    '''
    plt.plot(x,costs[0])
    plt.plot(x,costs[1], 'r')
    plt.plot(x,costs[2], 'g')
    plt.show()
    '''
    return thetaMatrix

In [45]:
def initTestData(Digits):
    yVector = Digits[Digits.columns[-1]]
    xMatrix = np.ones((Digits.columns[-1]+1,len(yVector)))
    classes = list(set(yVector))
    yMatrix = np.zeros((len(yVector),len(classes)))
    for i in range(len(classes)):
        for j in range(len(yVector)):
            if yVector[j] == classes[i]:
                yMatrix[j][i] = 1
            else:
                yMatrix[j][i] = 0
    yMatrix = yMatrix.T
    return xMatrix, yMatrix

In [54]:
def accuracyMeasure(theta, xMatrix, yVector):
    score = 0
    for i in range(len(yVector)):
        prediction = -math.log(hypothesis(theta,xMatrix[i]))
        if prediction > 0.5 and yVector[i] == 1:
            score +=1
    return score

In [53]:
# Compare logistic regression an naive Bayes

Lambda = 10
logregThetaMatrix = logreg(Training, Lambda)
xMatrix, yMatrix = initTestData(Test)
score = 0
for i in range(len(yMatrix)):
    score += accuracyMeasure(logregThetaMatrix[i], xMatrix.T, yMatrix[i])

print "Accuracy of logistic regression in percentage: ", score/len(yMatrix[0])*100.0

classDict = makeClassDict(TrainingNpArray)

allClasses = meanStdAll(classDict)

predicts = getPredictions(allClasses, TrainingNpArray[:,:-1])

print "Accuracy of naive Bayes in percentages: ", accuracy(predicts, TrainingNpArray[:,-1])*100.0



240
240
240
Accuracy of logistic regression in percentage:  100.0
Accuracy of naive Bayes in percentages:  98.6666666667


#### discussion

As shown above the percentages of both classifiers are quite high, the logistic regression even scores 100 percent.
So the logistic regression is slightly better.